# HK Stock Screening (Post Market) v0

In this notebook we try to achieve
- Introduce Nasdaq API
- Screen HK Common Stock with previous day Volume

Timeline as follow:

|| T0 | T1 | T2 | T3 |
| --- | --- | --- | --- | --- |
| Expected trend | x | Volume Rise | Volume Drop | Price Rise |
| Action | x | **Create Potential Stocks List** | Enter Market near day end | |

Import Library

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
import nasdaqdatalink
from datetime import date

Import Token

In [ ]:
from secrets import NASDAQ_API_TOKEN
nasdaqdatalink.ApiConfig.api_key = NASDAQ_API_TOKEN
SEND_TELEGRAM = False

Nasdaq Data Link API test call

In [ ]:
data = nasdaqdatalink.get('HKEX/06823',
#                           column_index=10,
                          rows=2,
                          order='asc',
#                           end_date="2022-07-07",
                         )
data

## Preparation

Fetch all available symbol from pre-generate csv [(Link)](https://data.nasdaq.com/data/HKEX-hong-kong-exchange/documentation?anchor=data-organization) .

In [ ]:
allHkSymbolsDataFrame = data = pd.read_csv(
    f'https://data.nasdaq.com/api/v3/databases/HKEX/metadata?api_key={NASDAQ_API_TOKEN}',
    compression='zip'
)
allHkSymbolsDataFrame

Filter all non common stock

In [ ]:
allHkSymbolsDataFrame[
    ('00000' ==  allHkSymbolsDataFrame['code']) |
    ('09999' < allHkSymbolsDataFrame['code'])
]

In [ ]:
allHkSymbolsDataFrame.drop(
    allHkSymbolsDataFrame[
        ('00000' ==  allHkSymbolsDataFrame['code']) |
        ('09999' < allHkSymbolsDataFrame['code'])
    ].index,
    inplace = True
)
allHkSymbolsDataFrame

Drop row that don't have latest data / have already stop update

In [ ]:
allHkSymbolsDataFrame[
    (allHkSymbolsDataFrame['to_date'] < str(date.today()))
]

In [ ]:
allHkSymbolsDataFrame.drop(
    allHkSymbolsDataFrame[
        (allHkSymbolsDataFrame['to_date'] < str(date.today()))
    ].index,
    inplace = True
)
allHkSymbolsDataFrame

## Data Fetching

Create main DataFrame, including volume of today, previous trading date.

In order to get the previous trading date, we need to call **/previous**, or else we can use **/quote/previousVolume** directly

Data imported from IEX Cloud API.

In [ ]:
%%time

potentialDataFrameColumns = [
    'Ticker',
    'Price',
    '30 day Average Volume',
    'T0 Date',
    'T0 Volume',
    'T1 Volume',
    'T1 Volume Ratio',
]

potentialDataFrame = pd.DataFrame(columns = potentialDataFrameColumns)

for hkSymbol in allHkSymbolsDataFrame['code']:
    data = nasdaqdatalink.get(f'HKEX/{hkSymbol}',
                              rows=2,
                              order='asc'
                             )
#     print(data)

    try:
        latestPrice = data.iloc[1]['Nominal Price']
    except (KeyError, IndexError):
        latestPrice = np.NaN

    try:
        t0Volume = data.iloc[0]['Share Volume (000)'] * 1000
    except (KeyError, IndexError):
        try:
            t0Volume = data.iloc[0]['Share Volume (\'000)'] * 1000
        except (KeyError, IndexError):
            t0Volume = np.NaN

    try:
        t1Volume = data.iloc[1]['Share Volume (000)'] * 1000
    except (KeyError, IndexError):
        try:
            t1Volume = data.iloc[1]['Share Volume (\'000)'] * 1000
        except (KeyError, IndexError):
            t1Volume = np.NaN
    
    potentialDataFrame = potentialDataFrame.append(
        pd.Series(
            [
                hkSymbol,
                latestPrice,
                'N/A',
                str(data.iloc[0].name.date()),
                t0Volume,
                t1Volume,
                'N/A'
            ],
            index = potentialDataFrameColumns
        ),
        ignore_index = True
    )

In [ ]:
potentialDataFrame

## Data Cleaning

Clean out data with None from API

In [ ]:
potentialDataFrame[potentialDataFrame.isnull().any(axis = 1)]
# potentialDataFrame[potentialDataFrame['Price'].isnull()]

In [ ]:
potentialDataFrame = potentialDataFrame.dropna()
potentialDataFrame

Drop data with 0 Volume

In [ ]:
potentialDataFrame[
    (potentialDataFrame['T0 Volume'] == 0) |
    (potentialDataFrame['T1 Volume'] == 0)
]

In [ ]:
potentialDataFrame.drop(
    potentialDataFrame[
        (potentialDataFrame['T0 Volume'] == 0) |
        (potentialDataFrame['T1 Volume'] == 0)
    ].index,
    inplace = True
)
potentialDataFrame

## Calculation

Calculate Volume Ratio

In [ ]:
for row in potentialDataFrame.index:
    potentialDataFrame.loc[row, 'T1 Volume Ratio'] = potentialDataFrame.loc[row, 'T1 Volume'] / potentialDataFrame.loc[row, 'T0 Volume']
    
potentialDataFrame

## Screening

Screening criteria are as follow:

- Average Volume can't be too low
- T1 Volume can't be too low
- T1 Volume is more than 5 times of T0

In [ ]:
volumeLowerBound = float(2e6)
t1traget = float(5)

potentialDataFrame = potentialDataFrame[
#     (volumeLowerBound < potentialDataFrame['30 day Average Volume']) &
#     (volumeLowerBound < potentialDataFrame['T1 Volume']) &
    (t1traget < potentialDataFrame['T1 Volume Ratio'])
]
potentialDataFrame.sort_values(
    'Ticker',
    ascending = True,
    inplace = True
)
potentialDataFrame.reset_index(inplace = True, drop = True)

potentialDataFrame

In [ ]:
','.join(potentialDataFrame['Ticker'].to_list())

## Add 30 days average

As Nasdaq API did not return 30-days-average, we have to calculate ourselves.

In [ ]:
for row in potentialDataFrame.index:
    symbol = potentialDataFrame.loc[row, 'Ticker']
    averageVolume = nasdaqdatalink.get(f'HKEX/{symbol}',
                                       column_index=10,
                                       rows=30,
                                       order='asc',
                                       end_date="2022-07-06",
                                      ).mean() * 1000
    potentialDataFrame.loc[row, '30 day Average Volume'] = float(averageVolume)

potentialDataFrame

## Data Export

For the ease of the next programe, we will also export the file in CSV version.

In [ ]:
from datetime import date
import os

today = date.today().strftime("%Y%m%d")
folderName = f'../export/{today}'
fileName = f'{folderName}/yau4muk6man4_strategy_HK_{today}_volume_rise'
csvFileName = f'{fileName}.csv'
xlsxFileName = f'{fileName}.xlsx'
sheetName = f'{today} Volume Rise'

In [ ]:
try:
    os.mkdir(folderName)
    print("Directory " , folderName ,  " Created ") 
except FileExistsError:
    print("Directory " , folderName ,  " already exists")

Export to csv

In [ ]:
potentialDataFrame.to_csv(csvFileName, index = False)

Export to xlsx

In [ ]:
writer = pd.ExcelWriter(xlsxFileName, engine = 'xlsxwriter')
potentialDataFrame.to_excel(writer, sheet_name = sheetName, index = False)

In [ ]:
backgroundColor = '#0a0a23'
fontColor = '#ffffff'

stringTemplate = writer.book.add_format(
        {
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

dollarTemplate = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

integerTemplate = writer.book.add_format(
        {
            'num_format':'#,###',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

floatTemplate = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

percentTemplate = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': fontColor,
            'bg_color': backgroundColor,
            'border': 1
        }
    )

In [ ]:
columnFormats = {
    'A': ['Ticker', stringTemplate],
    'B': ['Price', dollarTemplate],
    'C': ['30 day Average Volume', integerTemplate],
    'D': ['T0 Date', stringTemplate],
    'E': ['T0 Volume', integerTemplate],
    'F': ['T1 Volume', integerTemplate],
    'G': ['T1 Volume Ratio', percentTemplate],
}

for column in columnFormats.keys():
    writer.sheets[sheetName].set_column(
        f'{column}:{column}',
        max(len(columnFormats[column][0]), 10),
        columnFormats[column][1]
    )
    writer.sheets[sheetName].write(
        f'{column}1',
        columnFormats[column][0],
        columnFormats[column][1]
    )

In [ ]:
writer.save()

## Telegram

In [ ]:
import telegram
from secrets import TELEGRAM_CHAT_ID
from secrets import TELEGRAM_API_TOKEN

In [ ]:
telegramBot = telegram.Bot(TELEGRAM_API_TOKEN)
telegramBot

In [ ]:
txt = f'''
⬆️⬆️⬆️ {today} Volume Rise ⬆️⬆️⬆️

This is report of stock that have rise in volume during {today}.
We have found {len(potentialDataFrame.index)} stock(s) that match the screening requirements.

Rise Ticker:
{','.join(potentialDataFrame['Ticker'])}

Detail as in follow xlsx.
'''

output = open(xlsxFileName, 'rb')

In [ ]:
if SEND_TELEGRAM:
    telegramBot.send_message(TELEGRAM_CHAT_ID, txt)
    telegramBot.send_document(TELEGRAM_CHAT_ID, output)